## Find key; returns a partial tree

Not very useful

In [1]:
from typing import Any, Dict, List, Optional

def find_keys_tree(
    data: Any,
    key: Optional[str] = None,
    substring: Optional[str] = None
) -> List[Dict[str, Any]]:
    """
    Recursively search for keys in nested JSON-like structures and
    return results in a tree-style string format.

    Args:
        data: The object to search (dict, list, or scalar)
        key: Exact key to match (optional)
        substring: Substring to match in keys (optional)

    Returns:
        List[Dict[str, Any]]:
            Each entry: {
                "tree": "<tree path>",
                "value": <object>
            }
    """

    results: List[Dict[str, Any]] = []
    seen_paths: set[str] = set()

    def _search(obj: Any, path: str, depth: int) -> None:
        """
        Recursive helper for traversing the structure.
        """

        # Searching inside dictionaries
        if isinstance(obj, dict):
            for k, v in obj.items():
                new_path = f"{path}.{k}" if path else k

                # Match conditions
                matched: bool = False
                if key is not None and k == key:
                    matched = True
                if substring is not None and substring in k:
                    matched = True

                if matched and new_path not in seen_paths:
                    seen_paths.add(new_path)

                    # Build a pretty tree representation
                    tree_line = "│   " * (depth - 1) + ("└── " if depth > 0 else "") + k
                    results.append({
                        "tree": tree_line,
                        "value": v
                    })

                _search(v, new_path, depth + 1)

        # Searching inside lists
        elif isinstance(obj, list):
            for idx, item in enumerate(obj):
                new_path = f"{path}[{idx}]"
                label = f"[{idx}]"
                tree_line = "│   " * (depth - 1) + ("└── " if depth > 0 else "") + label

                # Lists themselves do not match keys, only elements inside
                _search(item, new_path, depth + 1)

        # Scalars do not recurse further

    _search(data, path="", depth=0)
    return results


In [2]:
data = {
    "user": {
        "name": "Alice",
        "address": {
            "street": "123 Road",
            "zipcode": 99999
        },
        "hobbies": [
            {"type": "sport", "name": "tennis"},
            {"type": "music", "name": "piano"}
        ]
    }
}

matches = find_keys_tree(data, key="name")


In [3]:
print(matches)

[{'tree': '└── name', 'value': 'Alice'}, {'tree': '│   │   └── name', 'value': 'tennis'}, {'tree': '│   │   └── name', 'value': 'piano'}]


In [4]:
for match in matches:
    print(match)

{'tree': '└── name', 'value': 'Alice'}
{'tree': '│   │   └── name', 'value': 'tennis'}
{'tree': '│   │   └── name', 'value': 'piano'}


In [8]:
with open("users.json") as file:
    data = json.load(file)

for index, user in enumerate(data):
    if index > 12:
        break
    results = find_keys_tree(user, "lat")

for result in results:
    print(result)
# print(result)
#     if len(result) > 0:
#         print(user)

{'tree': '│   └── lat', 'value': -25.0377}
{'tree': '│   │   └── lat', 'value': 19.490447}


In [10]:
data1 = {
    "user": {
        "profile": {
            "name": "Alice",
            "details": {
                "bo": [[
                    {"age": 30, "location": {"city": "Boston"}},
                    {"projects": [{"title": "X"}, {"title": "Y", "city": "Miami"}]}
                ]]}
        }
    }
}
results = find_keys_tree(data1, "city")
print(results)
# for result in results:
#     print(f"{result['path']=}")
#     print(f"{result['value']=}")

[{'tree': '│   │   │   │   │   │   └── city', 'value': 'Boston'}, {'tree': '│   │   │   │   │   │   │   └── city', 'value': 'Miami'}]
